## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unicodedata
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://www.mendocinocounty.org/community/novel-coronavirus/covid-19-case-data"

In [7]:
url = "https://www.mendocinocounty.org/community/novel-coronavirus/covid-19-case-data/covid-dashboard-draft/cases-and-deaths-demographics"

In [8]:
page = requests.get(url)

##  Parse

In [9]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [10]:
content = soup.find("div", {"id": "widget_4_7164_4508"})

In [11]:
content = soup.find_all("iframe")

In [12]:
source = content[1]["src"]

In [13]:
newPage = requests.get(source)

In [14]:
newSoup = BeautifulSoup(newPage.content, "html.parser")

In [15]:
src = newSoup.find("meta")["content"].split("=")[1]

In [16]:
page2 = requests.get(src)

In [17]:
soup2 = BeautifulSoup(page2.content, "html.parser")

In [18]:
src

'https://datawrapper.dwcdn.net/zj83H/9/'

Get timestamp

In [19]:
date_text = soup2.find("div", {"class": "dw-above-footer"}).text

In [20]:
date_text

'\nData as of 3/30/22 /Datos al 30/03/22. \nUkiah area / Área de Ukiah: zip codes 95418, 95449, 95469, 95470, 95481, 95482.\nNorth County / Condado Norte: zip codes 95417, 95428, 95429, 95454, 95490, 95585, 95587. \nNorth Coast / Costa Norte: zip codes 95420, 95437, 95488. South County / Condado Sur: zip codes 95414, 95463, 95466, 95494. South Coast / Costa Sur: 95410, 95427, 95432, 95445, 95456, 95459, 95460, 95468. \n'

In [21]:
assert "Data as of" in date_text

In [22]:
date = date_text.split("/Datos")[0].replace("\nData as of ", "")

In [23]:
date

'3/30/22 '

In [24]:
latest_date = pd.to_datetime(date).date()

Get table

In [25]:
table = soup2.find("table")

In [26]:
soup2

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="noindex, indexifembedded, nofollow" name="robots"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="light dark" name="color-scheme"/><link href="../../lib/vis/tables.datawrapper-data.7ae92e8b.css" id="css-light" media="(prefers-color-scheme: light)" rel="stylesheet"/><link href="../../lib/vis/tables.datawrapper-data-dark.81e259da.css" id="css-dark" media="(prefers-color-scheme: dark)" rel="stylesheet"/><style id="hide-all">* { opacity: 0 }</style>
<link href="https://api.datawrapper.de/v3/oembed?url=https://datawrapper.dwcdn.net/zj83H/9/&amp;format=json" rel="alternate" title="oEmbed" type="application/json+oembed">
<meta content="https://datawrapper.dwcdn.net/zj83H/plain-s.png?v=9" property="og:image"/>
</link></head><body><div class="dw-chart chart vis-height-fixed theme-datawrapper-data vis-tables" id="__svelte-dw"><div class="dw-chart-header" id="header"><h1 class="block

In [27]:
row_list = tbody.find_all("tr")[1:]

NameError: name 'tbody' is not defined

In [ ]:
dict_list = []

In [ ]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [ ]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [ ]:
for row in row_list:
    cell_list = row.find_all("td")
    d = dict(
        county="Mendocino",
        area=safetxt(cell_list[0]),
        confirmed_cases=safenumber(cell_list[1]),
    )
    dict_list.append(d)

In [ ]:
df = pd.DataFrame(dict_list)

In [ ]:
df["county_date"] = latest_date

## Vet

In [ ]:
try:
    assert not len(df) > 5
except AssertionError:
    raise AssertionError("Mendocino's area scraper has extra rows")

In [ ]:
try:
    assert not len(df) < 5
except AssertionError:
    raise AssertionError("Mendocino's area scraper is missing rows")

## Export

Set date

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
slug = "mendocino"

In [ ]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [ ]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [ ]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)